*Acknowledgements. Данное занятие в значительной степени полагается на материалы семинара Антона Михайловича Алексеева, который он провёл на Летней школе 2018 года.*

In [2]:
with open("data/автостопом.txt", encoding="cp1251") as f:
    txt = f.read()

In [3]:
txt[:200]

'Посвящается Джонни Броку, Клэр Горст и всем остальным арглингтонийцам — в благодарность за чай, сочувствие и диван.\n\n\nДалеко-далеко, в не замеченных картографами складках давно вышедшего из моды Запад'

In [4]:
import re
from collections import Counter

BOS = "BOS"
EOS = "EOS"
UNK = "UNK"

def prepare_sentences(txt, word_threshold=2, stage_train=True):
    
    # заменяем многие пробелы на один
    txt = re.sub("\s+", " ", txt) \
        .lower() # и переводим в нижний регистр
    
    # заменяем символы переноса строки на пробелы (абзацы нам не нужны)
    text = txt.replace("\n", " ")
    # разбиваем текст на предложения по знакам препинания
    sentences = re.split("[!\?\.]+", txt.replace("\n", " "))
    
    # оставляем только alphanumeric
    # W — любой символ, кроме буквенного или цифрового символа или знака подчёркивания
    clean_sentences = [re.split("\W+", s) for s in sentences]
    
    # заменяем числа на NUM
    clean_sentences = [[w.replace("\d+", "NUM") for w in s if w] for s in clean_sentences]
    
    # вводим тег UNKNOWN: UNK
    if stage_train:

        counter = Counter()

        for s in clean_sentences:
            for w in s:
                counter[w] += 1
    
        print("Filtered out word types :", len([w for w in counter if counter[w] <= word_threshold]))
        print("Filtered out words count:", sum([counter[w] for w in counter if counter[w] <= word_threshold]))
    
        # выкидываем редкие, и заменяем их на специальный тег
        clean_sentences = [[w if counter[w] > word_threshold else UNK for w in s] for s in clean_sentences]            
    
    word2index = { BOS: 0, EOS: 1, UNK: 2}
    index2word = { 0: BOS, 1: EOS, 2: UNK}
    
    counter = max(word2index.values()) + 1

    for s in clean_sentences:
        for w in s:
            if not w in word2index:
                word2index[w] = counter
                index2word[counter] = w
                counter += 1
                
    return word2index, index2word, clean_sentences

In [5]:
word2index, index2word, clean_sentences = prepare_sentences(txt)

print("Total number of sentences :\t", len(clean_sentences))
print("Total number of words     :\t", sum([len(sent) for sent in clean_sentences]))
print("Total number of word types:\t", len(set([w for sent in clean_sentences for w in sent])))

Filtered out word types : 34282
Filtered out words count: 41806
Total number of sentences :	 25610
Total number of words     :	 275830
Total number of word types:	 12238


In [6]:
def augment(sentence, context_size):
    """
        Добиваем символы начала и конца строки к каждому предложению
    """
    return [BOS] * context_size + sentence + [EOS] * context_size

def enumerate_sentences(clean_sentences, context_size, word2index):
    """
        Добиваем символами начала и конца и конвертируем слова в индексы
    """

    contexts = []
    targets = []
    UNK_id = word2index[UNK]

    for sentence in clean_sentences:

        aligned_sentence =  augment(sentence, context_size) 

        for i in range(context_size, len(sentence) - context_size, 1):
            
            # берём предшествующий контекст
            context = aligned_sentence[i - context_size:i]
            context = [word2index[c] if c in word2index else UNK_id for c in context]
            target = word2index[aligned_sentence[i]] if aligned_sentence[i] in word2index else UNK_id
            
            contexts.append(context)
            targets.append(target)
    
    return contexts, targets

In [7]:
def chunks(l0, l1, n):
    
    assert len(l0) == len(l1)
    coll0, coll1 = [], []
    
    for i in range(0, len(l0), n):
        coll0.append(l0[i:i + n])
        coll1.append(l1[i:i + n])
        
    return coll0, coll1

In [8]:
from collections import defaultdict
from tqdm import tqdm_notebook
from functools import lru_cache

class NGramFreqsLanguageModeler(object):
    
    def __init__(self, vocab_size, context_size):
        super(NGramFreqsLanguageModeler, self).__init__()
    
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.ngram_dict = defaultdict(lambda: defaultdict(lambda: 0))        
        self.n_1_gram_dict = defaultdict(lambda: defaultdict(lambda: 0))
        self.contexts_counts = defaultdict(lambda: 0)
        self.eps = 1.0
    
    def fit(self, contexts, targets):
        
        self.contexts_counts = defaultdict(lambda: 0)
        
        for c, t in zip(contexts, targets):
            c = tuple(c)
            self.ngram_dict[c][t] += 1
            self.contexts_counts[c] += 1
            
            # намёк!
            # self.n_1_gram_dict[c[1:]][t] += 1

            
        print("Total n-1 grams", len(self.ngram_dict), list(self.ngram_dict)[:10])
        
        # нормализуем частоты
        for c in tqdm_notebook(self.ngram_dict.keys()):
            for t in self.ngram_dict[c]:
                self.ngram_dict[c][t] = (self.ngram_dict[c][t] +  self.eps) / \
                                            (self.contexts_counts[c] + self.vocab_size * self.eps)
        
    @lru_cache(1000000)
    def prob_dist(self, input_context):
        """
            Takes ngram as a tuple
        """
        
        probs = np.zeros(self.vocab_size) + \
                    self.eps / (self.vocab_size * self.eps + self.contexts_counts[input_context])
        
        counts = self.ngram_dict[input_context]
        
        # если есть хоть какие-то счётчики
        if counts:
            
            # проставим осмысленные частоты
            for target, freq in counts.items():
                probs[target] = freq
                
        return probs

In [9]:
CONTEXT_SIZE = 3
BATCH_SIZE = 2048

In [10]:
from tqdm import tqdm_notebook
import numpy as np

# строим контексты и цели
contexts, targets = enumerate_sentences(clean_sentences, CONTEXT_SIZE, word2index)

batches = list(zip(contexts, targets))

simple_model = NGramFreqsLanguageModeler(context_size=CONTEXT_SIZE, vocab_size=len(word2index))
simple_model.fit(contexts, targets)

Total n-1 grams 101108 [(0, 0, 0), (0, 0, 3), (0, 3, 4), (3, 4, 2), (4, 2, 2), (2, 2, 2), (2, 2, 5), (2, 5, 6), (5, 6, 7), (6, 7, 2)]


In [11]:
test = "BOS"
prepared_text = augment(prepare_sentences(test, stage_train=False)[2][0], CONTEXT_SIZE)[-CONTEXT_SIZE:]

for i in range(CONTEXT_SIZE, 10 + CONTEXT_SIZE):
    
    idx = [word2index[w] for w in prepared_text[:i]]    
    
    predict = simple_model.prob_dist(tuple(idx[-CONTEXT_SIZE:])) 
    
#     predict = predict - predict.min()  
#     predict /= sum(predict)
    
    selected_word = np.random.choice(a=list(range(len(word2index))), p=predict)    
    prepared_text.append(index2word[selected_word])
    
    print("Генерация:", " ".join(prepared_text[CONTEXT_SIZE - 1:]))

Генерация: EOS который
Генерация: EOS который ночей
Генерация: EOS который ночей блестящий
Генерация: EOS который ночей блестящий доброго
Генерация: EOS который ночей блестящий доброго мистер
Генерация: EOS который ночей блестящий доброго мистер кататься
Генерация: EOS который ночей блестящий доброго мистер кататься алфавит
Генерация: EOS который ночей блестящий доброго мистер кататься алфавит ярдах
Генерация: EOS который ночей блестящий доброго мистер кататься алфавит ярдах странности
Генерация: EOS который ночей блестящий доброго мистер кататься алфавит ярдах странности отбросил


In [12]:
with open("data/ресторан.txt") as f:
    test_txt = f.read()

_, _, test_clean_sentences = prepare_sentences(test_txt, stage_train=False)

print("Total number of sentences :\t", len(test_clean_sentences))
print("Total number of words     :\t", sum([len(sent) for sent in test_clean_sentences]))
print("Total number of word types:\t", len(set([w for sent in test_clean_sentences for w in sent])))

# строим контексты и цели
test_contexts, test_targets = enumerate_sentences(test_clean_sentences, CONTEXT_SIZE, word2index)

# test_data = list(zip(test_contexts, test_targets))

test_batched_contexts, test_batched_targets = chunks(test_contexts, test_targets, BATCH_SIZE)
test_batches = list(zip(test_batched_contexts, test_batched_targets))

len(test_contexts), len(test_targets), len(test_batches), len(word2index)

Total number of sentences :	 4818
Total number of words     :	 45413
Total number of word types:	 11278


(22410, 22410, 11, 12240)

In [13]:
import torch
import tqdm
from tqdm import tqdm_notebook

def compute_ppl_count_model(model, test_batches, loss_function):
    
    total_loss = 0
    count = 0

    for context_batch, target_batch in tqdm_notebook(test_batches):
        
        log_probs = []
        
        for context, target in zip(context_batch, target_batch):
            
            # применяем модель
            log_probs.append(np.log2(model.prob_dist(tuple(context))))
            
        log_probs = np.array(log_probs)
        
        # вычисляем невязку
        loss = loss_function(torch.tensor(log_probs, dtype=torch.float), 
                       torch.tensor(target_batch, dtype=torch.long))
        
        # получаем число
        total_loss += loss.item()
        count += 1
        
        if count % (len(test_batches) // 5) == 0:
            print(count, "\tloss", total_loss)
            print([index2word[i] for i in context_batch[0]], "->", 
                  index2word[target_batch[0]], "vs", 
                  [index2word[i] for i in (-log_probs[0]).argsort()[:3]],
                  np.sort((-log_probs[0]))[:3]
                 )
    
    return 2 ** (total_loss / count)

In [14]:
from torch.nn import CrossEntropyLoss

print("Perplexity of freq-based NGram model on test set", compute_ppl_count_model(model=simple_model, 
                                                                            loss_function=CrossEntropyLoss(),
                                                                            test_batches=test_batches))

2 	loss 17.510966300964355
['смотрела', 'прямо', 'на'] -> них vs ['BOS', 'здоровой', 'возбуждение'] [13.57931594 13.57931594 13.57931594]
4 	loss 34.798720359802246
['обращается', 'голос', 'UNK'] -> хозяина vs ['BOS', 'здоровой', 'возбуждение'] [13.57931594 13.57931594 13.57931594]
6 	loss 52.32221984863281
['во', 'время', 'UNK'] -> играют vs ['посадки', 'UNK', 'так'] [12.57990515 12.57990515 12.57990515]
8 	loss 69.98028659820557
['в', 'атмосфере', 'и'] -> все vs ['BOS', 'здоровой', 'возбуждение'] [13.57931594 13.57931594 13.57931594]
10 	loss 87.3246488571167
['UNK', 'гладь', 'UNK'] -> по vs ['BOS', 'здоровой', 'возбуждение'] [13.57931594 13.57931594 13.57931594]

Perplexity of freq-based NGram model on test set 426.5119103413951


# Задача
Требуется реализовать языковую модель, с помощью которой можно решать задачу отделения "естественных" текстов от сгенерированных -- с использованием предложенного обучающего множества.

Перед вами список пар текстов, и требуется сказать, какой из них "настоящий"; 1 — первый, 0 — второй. Оценка качества алгоритма -- accuracy, то есть доля истинных попаданий.

Модель обучается на `train.tsv`. Модель применяется к `test.tsv`.

Истинные метки можно найти в файле `true_labels.tsv`.

Можно использовать вышеопределённую модель или любую другую.